In [1]:
!pip install datasets

In [2]:
!pip install datasets transformers torch

  Using cached tbb-2021.13.1-py3-none-win_amd64.whl.metadata (1.1 kB)
Using cached tbb-2021.13.1-py3-none-win_amd64.whl (286 kB)
  Attempting uninstall: tbb
    Found existing installation: tbb 2022.0.0
    Uninstalling tbb-2022.0.0:
      Successfully uninstalled tbb-2022.0.0


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.1.0+cu121 requires torch==2.1.0+cu121, but you have torch 2.3.0 which is incompatible.
torchsig 0.5.0 requires opencv-python==4.8.0.74, but you have opencv-python 4.7.0 which is incompatible.


In [3]:
!python -m spacy download ru_core_news_sm

     ---------------------------------------- 0.0/15.3 MB ? eta -:--:--
     ---------------------------------------- 0.0/15.3 MB ? eta -:--:--
      --------------------------------------- 0.3/15.3 MB ? eta -:--:--
     - -------------------------------------- 0.5/15.3 MB 2.8 MB/s eta 0:00:06
     -- ------------------------------------- 0.8/15.3 MB 1.3 MB/s eta 0:00:12
     -- ------------------------------------- 1.0/15.3 MB 1.3 MB/s eta 0:00:11
     ---- ----------------------------------- 1.6/15.3 MB 1.6 MB/s eta 0:00:09
     ----- ---------------------------------- 2.1/15.3 MB 1.7 MB/s eta 0:00:08
     ------- -------------------------------- 2.9/15.3 MB 2.0 MB/s eta 0:00:07
     --------- ------------------------------ 3.7/15.3 MB 2.2 MB/s eta 0:00:06
     ---------- ----------------------------- 4.2/15.3 MB 2.3 MB/s eta 0:00:05
     ------------- -------------------------- 5.0/15.3 MB 2.5 MB/s eta 0:00:05
     --------------- ------------------------ 5.8/15.3 MB 2.5 MB/s eta 0:

In [6]:
from typing import List, Tuple
import numpy as np
import torch
import tqdm
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from datasets import load_dataset
from transformers import BertTokenizer, BertModel

In [7]:
def load_sib200_ru() -> Tuple[Tuple[List[str], List[int]], Tuple[List[str], List[int]], Tuple[List[str], List[int]], List[str]]:
    trainset = load_dataset('Davlan/sib200', 'rus_Cyrl', split='train')
    X_train = trainset['text']
    y_train = trainset['category']
    valset = load_dataset('Davlan/sib200', 'rus_Cyrl', split='validation')
    X_val = valset['text']
    y_val = valset['category']
    testset = load_dataset('Davlan/sib200', 'rus_Cyrl', split='test')
    X_test = testset['text']
    y_test = testset['category']
    categories = set(y_train)
    unknown_categories = set(y_val) - categories
    if len(unknown_categories) > 0:
        err_msg = f'The categories {unknown_categories} are represented in the validation set, but they are not represented in the training set.'
        raise RuntimeError(err_msg)
    unknown_categories = set(y_test) - categories
    if len(unknown_categories) > 0:
        err_msg = f'The categories {unknown_categories} are represented in the test set, but they are not represented in the training set.'
        raise RuntimeError(err_msg)
    categories = sorted(list(categories))
    y_train = [categories.index(it) for it in y_train]
    y_val = [categories.index(it) for it in y_val]
    y_test = [categories.index(it) for it in y_test]
    return (X_train, y_train), (X_val, y_val), (X_test, y_test), categories

In [8]:
# БЕРТ для векторизации
class BertVectorizer:
    def __init__(self):
        self.tokenizer = BertTokenizer.from_pretrained('DeepPavlov/rubert-base-cased')
        self.model = BertModel.from_pretrained('DeepPavlov/rubert-base-cased')

    def transform(self, texts: List[str]) -> np.ndarray:
        inputs = self.tokenizer(texts, return_tensors='pt', padding=True, truncation=True)
        with torch.no_grad():
            outputs = self.model(**inputs)
        return outputs.last_hidden_state[:, 0, :].numpy()
train_data, val_data, test_data, classes_list = load_sib200_ru()
vectorizer = BertVectorizer()
X_train_vec = vectorizer.transform(train_data[0])
X_val_vec = vectorizer.transform(val_data[0])
X_test_vec = vectorizer.transform(test_data[0])

tokenizer_config.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

C:\Users\snytk\miniconda3\envs\python311\Lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\snytk\.cache\huggingface\hub\models--DeepPavlov--rubert-base-cased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.txt:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum lengt

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

In [9]:
classifier = LogisticRegression(solver='saga', max_iter=50, random_state=42) #убрал грид
classifier.fit(X_train_vec, train_data[1])

C:\Users\snytk\miniconda3\envs\python311\Lib\site-packages\sklearn\linear_model\_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


LogisticRegression(max_iter=50, random_state=42, solver='saga')

In [10]:
y_pred = classifier.predict(X_val_vec)
print(classification_report(val_data[1], y_pred, target_names=classes_list))

                    precision    recall  f1-score   support

     entertainment       0.62      0.56      0.59         9
         geography       0.62      0.62      0.62         8
            health       1.00      0.91      0.95        11
          politics       0.82      1.00      0.90        14
science/technology       0.89      1.00      0.94        25
            sports       1.00      0.92      0.96        12
            travel       0.65      0.55      0.59        20

          accuracy                           0.82        99
         macro avg       0.80      0.79      0.79        99
      weighted avg       0.81      0.82      0.81        99

